<br>

# Introdução

In [ ]:
import os
import pprint
import shutil
import fnmatch
import pandas as pd
import geopandas as gpd

In [ ]:
from osgeo import ogr
from folium import plugins
from zipfile import ZipFile

In [ ]:
from traquitanas.geo import layers as tt

In [ ]:
from paths import *

<br>

Inicialmente extraímos todos os arquivos de dentro do zip "Shape.zip".

In [ ]:
shutil.rmtree(temp_path, ignore_errors=True)

In [ ]:
zipfile_path = input_path / 'Shapes.zip'

with ZipFile(zipfile_path, 'r') as zip_obj:    
    zip_obj.extractall(temp_path)

<br>

Uma vez com os arquivos soltos, foi possível observar grande desorganização dos arquivos. Não há qualquer padrão de nomenclatura. Alguns *shapefiles* existem para algumas nascentes, enquanto para outra snão existe.<br>
Como o objetivo principal do trabalho é identificar a localização das 56 nascentes que são mencionadas no TAC, optou-se por concentrar os esforços nestas feições.

In [ ]:
list_shps = list(set([x.name for x in list(temp_path.glob('Nascente*/**/*.shp'))]))
list_shps.sort()
pprint.pprint(list_shps)

In [ ]:
list_files = list(set([x.name for x in list(temp_path.glob(f'Nascente*'))]))
list_files

<br>

## Junta shapefiles por Nome

Com a lista dos possíveis nomes de shapefiles, juntou-se todos os arquivos.

In [ ]:
def merge_shp_by_name(shp, in_path, out_path):
    """
    in_path: pasta root que tem os shapefiles
    out_path: pasta que salva
    
    """
    # Create List Shapefiles
    list_files = list(set([x for x in list(in_path.glob(f'Nascente*/**/{shp}'))]))
        
    # Combine all shapefiles
    gdf = gpd.GeoDataFrame(
        pd.concat(
            [gpd.read_file(i) for i in list_files],
            ignore_index=True
        ),
        crs=gpd.read_file(list_files[0]).crs
    )
    
    # ddd
    #print(gdf.head())
    #print(gdf.columns)

    # Adjust Columns
    gdf.columns = [x.lower() for x in gdf.columns]
    
    # Delete Fields
    gdf.drop(
        [
            'area',
            'id',
            'in_path',
            'numero',
            'x',
            'y',
            'buff_dist',
            'nascente_1'
            'nascente_1',
            'nascente_2',
            'nascente_3',
            'nascente_4',
            'nascente_5',
            'na',
            'uso',
        ],
        axis=1,
        errors='ignore',
        inplace=True,
    )

    # Save Output
    geopackage = shp.replace('shp', 'gpkg')
    geopackage_filepath = out_path / f'{geopackage}'
    geopackage_filepath.unlink(missing_ok=True)
    gdf.to_file(
        geopackage_filepath,
        driver='GPKG',
    )    
    return gdf

In [ ]:
for shp in list_shps:
    print(shp)
    
    # Add Fields
    for i in temp_path.glob(f'Nascente*/**/{shp}'):
        #print(i)
        # Read Data
        gdf = gpd.read_file(i)

        # Delete Column
        gdf.drop(
            ['nascente','Nascente', 'path'],
            axis=1,
            errors='ignore',
            inplace=True,
        )
        # Add Fields
        gdf['nascente'] = str(i.parent.name)
        gdf['path'] = str(i.parent)

        # Save Data
        gdf.to_file(i)
    
    # Merge Data
    gdf = merge_shp_by_name(shp, temp_path, geo_path)

<br>

## Junta shapefiles Específicos

Após juntar todos os shapefiles em pastas, notei a existência de shapefiles que tem o mesmo tema, porém continham nomes distintos (por exemplo, "app.shp" e "App.shp").<br>
Esse problema na nomenclatura dos arquivos, impediu que os arquivos fossem unificados e, portanto, foi necessário escrever uma função que juntava os shapefiles, mantendo apenas o primeiro e deletando os restantes.

In [ ]:
def merge_shp_specific(list_files):
    """
    Une shapefiles específicos, sobrepondo o primeiro da lista...
    Deletando os demais...
    """
    # Concatena
    gdf = gpd.GeoDataFrame(
        pd.concat(
            [gpd.read_file(i) for i in list_files],
            ignore_index=True
        ),
        crs=gpd.read_file(list_files[0]).crs
    )
    
    # Delete
    for file in list_files[1:]:
        if file.exists:
            file.unlink()
            print(f'"{file}" deletado!!')
        else:
            print(f'"{file}" não existe!')    

    # Save Output
    gdf.to_file(os.path.join(list_files[0]))
    return gdf.explore()

In [ ]:
extension = 'gpkg'

In [ ]:
merge_shp_specific(
    [
        geo_path / f'APP.{extension}',
        geo_path / f'APP_nascente.{extension}',
    ]
)

In [ ]:
merge_shp_specific(
    [
        geo_path / f'Area_conservacao.{extension}',
        geo_path / f'area_Conservacao.{extension}',
    ]
)

In [ ]:
merge_shp_specific(
    [
        geo_path / f'area_drenagem.{extension}',
        geo_path / f'Area_Drenagem.{extension}',
        geo_path / f'Area_drenagem.{extension}',
    ]
)

<br>

## Deleta Shapefiles

Após juntar o material que considerei útil para a etapa do meu trabalho, foi possível deletar o restante do material.

In [ ]:
shutil.rmtree(temp_path, ignore_errors=True)